In [56]:
from matplotlib import pyplot as plt
import numpy as np
from collections import Counter
import cv2
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms

In [59]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    ])
train_dataset = datasets.Flowers102('data', split = 'train', download=True, transform=transform)
valid_dataset = datasets.Flowers102('data', split = 'val', download=True, transform=transform)
test_dataset = datasets.Flowers102('data', split = 'test', download=True,transform=transform)

In [60]:
def visualize(image, label):
  image = image.permute(1, 2, 0)
  image = image.numpy()
  plt.figure()
  plt.imshow(image)
  plt.title(str(label))
  plt.xticks([])
  plt.yticks([])


In [61]:
train_dataset[0][0].shape

torch.Size([3, 224, 224])

In [62]:
print(train_dataset[55][0].shape)

torch.Size([3, 224, 224])


In [ ]:
train_examples = [train_dataset[i] for i in range(10)]
for image, label in train_examples:
  visualize(image, f'Train: {label}')

In [ ]:
train_examples = [valid_dataset[i] for i in range(10)]
for image, label in train_examples:
  visualize(image, f'Test: {label}')


In [ ]:
test_examples = [test_dataset[i] for i in range(10)]
for image, label in test_examples:
  visualize(image, label)

In [ ]:
training_labels  = [train_dataset[i][1] for i in range(len(train_dataset))]
set(training_labels)

In [ ]:
test_labels  = [test_dataset[i][1] for i in range(len(test_dataset))]
set(test_labels)

In [ ]:
Counter(training_labels)

In [ ]:
Counter(test_labels)

In [70]:

train_image_sizes = [train_dataset[i][0].shape for i in range(len(train_dataset))]
set(train_image_sizes)

{torch.Size([3, 224, 224])}

In [71]:
test_image_sizes = [test_dataset[i][0].shape for i in range(len(test_dataset))]
set(test_image_sizes)

{torch.Size([3, 224, 224])}

## You can alternatively define the model as an object.
Note that you need to define the __init__ method and forward method. PyTorch takes care of the backward method for you.

In [174]:
class AlexNet(nn.Module):
    def __init__(self, num_channels, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=3)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(in_features=6*6*256, out_features=4096)
        self.linear2 = nn.Linear(in_features=4096, out_features=4096)
        self.linear3 = nn.Linear(in_features=4096, out_features=num_classes)

    def forward(self, x):
      x = self.conv1(x)
      print(x.shape)
      x = self.maxpool(x)
      x = self.relu(x)

      x = self.conv2(x)
      x = self.maxpool(x)
      x = self.relu(x)

      x = self.conv3(x)
      x = self.relu(x)

      x = self.conv4(x)
      x = self.relu(x)

      x = self.conv5(x)
      x = self.relu(x)

      x = self.maxpool(x)

      x = self.flatten(x)
      x = self.linear1(x)
      x = self.relu(x)

      x = self.linear2(x)
      x = self.relu(x)

      x = self.linear3(x)

      return x

In [177]:
class AlexNet(nn.Module):
    def __init__(self, num_channels, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4, padding=3),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=6*6*256, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes)
        )


    def forward(self, x):
      x = self.features(x)
      x = self.flatten(x)
      x = self.classifier(x)
      return x

In [178]:
model = AlexNet(num_channels=3, num_classes=102)

In [179]:
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

In [180]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [181]:
def evaluation(model, loader, criterion, device, phase='Valid'):
  model.eval()
  predictions = []
  ground_truth = []
  with torch.no_grad():
    total_loss = 0
    total  = 0
    correct = 0
    for batch_index, (images, labels) in enumerate(loader):
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, preds = torch.max(outputs, 1)
      predictions.extend(preds.cpu().numpy())
      ground_truth.extend(labels.cpu().numpy())
      correct += (preds == labels).sum().item()
    accuracy = correct / total
    loss = total_loss / total
    print(f'     {phase} Accuracy={accuracy:<10.4f}  Loss= {loss:<10.4f}')
    return {'loss': loss,
            'accuracy': accuracy,
            'ground_truth': ground_truth,
            'predictions': predictions}

In [182]:
def training(model, train_loader, valid_loader, criterion, optimizer, device,
             epochs, best_model_path):
  model.train()
  best_loss = torch.inf
  best_restults = None
  for epoch in range(epochs):
    total_loss = 0
    total  = 0
    correct = 0
    for batch_index, (images, labels) in enumerate(train_loader):
      optimizer.zero_grad()
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, preds = torch.max(outputs, 1)
      correct += (preds == labels).sum().item()
    accuracy = correct / total
    loss = total_loss / total
    print(f'{epoch:<4} Train Accuracy={accuracy:<10.4f}  Loss= {loss:<10.4f}')
    results = evaluation(model, valid_loader, criterion, device)
    if results['loss'] < best_loss:
      torch.save(model, best_model_path)
      best_loss = results['loss']
      best_restults = results
    print()
  return best_restults

In [183]:
epochs = 100
best_model_path = 'best_model.pt'
best_restults = training(model, train_loader, valid_loader, criterion,
                         optimizer, device, epochs, best_model_path)

0    Train Accuracy=0.0088      Loss= 4.6254    
     Valid Accuracy=0.0137      Loss= 4.6250    

1    Train Accuracy=0.0078      Loss= 4.6254    
     Valid Accuracy=0.0118      Loss= 4.6250    

2    Train Accuracy=0.0088      Loss= 4.6254    
     Valid Accuracy=0.0127      Loss= 4.6250    

3    Train Accuracy=0.0078      Loss= 4.6253    
     Valid Accuracy=0.0118      Loss= 4.6249    

4    Train Accuracy=0.0098      Loss= 4.6253    
     Valid Accuracy=0.0098      Loss= 4.6249    

5    Train Accuracy=0.0098      Loss= 4.6253    
     Valid Accuracy=0.0098      Loss= 4.6249    

6    Train Accuracy=0.0098      Loss= 4.6253    
     Valid Accuracy=0.0098      Loss= 4.6248    

7    Train Accuracy=0.0098      Loss= 4.6252    
     Valid Accuracy=0.0098      Loss= 4.6248    

8    Train Accuracy=0.0108      Loss= 4.6251    
     Valid Accuracy=0.0186      Loss= 4.6248    

9    Train Accuracy=0.0088      Loss= 4.6251    
     Valid Accuracy=0.0157      Loss= 4.6247    

10   Train

## Testing the last model


In [184]:
results = evaluation(model, test_loader, criterion, device, 'Test')

     Test Accuracy=0.1680      Loss= 7.7834    


## Testing the best model


In [186]:
best_model = torch.load(best_model_path)
results = evaluation(best_model, test_loader, criterion, device, 'Test')

     Test Accuracy=0.0859      Loss= 3.8630    
